# Challenge Set 9
## Part IV: Tennis Data

*Intermediate - Advanced level SQL*

---

### Acquire data ###

Let's get some data and start playing with it!

We'll be using tennis data from [here](https://archive.ics.uci.edu/ml/datasets/Tennis+Major+Tournament+Match+Statistics).


Assuming you are working on your AWS instance, execute the following from the BASH (shell) terminal:

```bash
mkdir -p tennis/data
cd tennis/data
wget http://archive.ics.uci.edu/ml/machine-learning-databases/00300/Tennis-Major-Tournaments-Match-Statistics.zip
```

You may not have `unzip` installed on your AWS instance:

```bash
sudo apt-get install unzip
unzip Tennis-Major-Tournaments-Match-Statistics.zip
```

Before we start using SQL, open up the files with your favorite command line text editor and poke around!

### Prepare the data ###

The data has a mix of missing entries and entries that are the string `NA`. We'll use the command line program `sed` to fix that:

```bash
sed -i.bak s/NA//g AusOpen-women-2013.csv
```

Repeat this for each CSV file in the data set.

### Import the data into PostgreSQL ###

Now we start working in PostgreSQL. The remaining commands are executed from the SQL prompt.

You can create a new database with `CREATE DATABASE tennis;` -- if we use `\d`, we see there are not yet any tables (relations) in the database. So let's create one.

We have to specify the schema of our table, with detail about [data types](http://www.postgresql.org/docs/9.3/static/datatype.html).

```sql
CREATE TABLE  aus_ladies_2013 (
      player1 VARCHAR(255),
      player2 VARCHAR(255),
      round INT,
      result INT,
      fnl1 DOUBLE PRECISION,
      fnl2 DOUBLE PRECISION,
      fsp_1 DOUBLE PRECISION,
      fsw_1 DOUBLE PRECISION,
      ssp_1 DOUBLE PRECISION,
      ssw_1 DOUBLE PRECISION,
      ace_1 INT,
      dbf_1 INT,
      wnr_1 INT,
      ufe_1 INT,
      bpc_1 INT,
      bpw_1 INT,
      npa_1 INT,
      npw_1 INT,
      tpw_1 INT,
      st1_1 INT,
      st2_1 INT,
      st3_1 INT,
      st4_1 INT,
      st5_1 INT,
      fsp_2 DOUBLE PRECISION,
      fsw_2 DOUBLE PRECISION,
      ssp_2 DOUBLE PRECISION,
      ssw_2 DOUBLE PRECISION,
      ace_2 INT,
      dbf_2 INT,
      wnr_2 INT,
      ufe_2 INT,
      bpc_2 INT,
      bpw_2 INT,
      npa_2 INT,
      npw_2 INT,
      tpw_2 INT,
      st1_2 INT,
      st2_2 INT,
      st3_2 INT,
      st4_2 INT,
      st5_2 INT);
```

Now load data from a CSV file into this table:

```sql
COPY 
      aus_ladies_2013
FROM 
      '/home/my_user_name/tennis/data/AusOpen-women-2013.csv'
DELIMITER 
      ','
CSV HEADER;
```

Repeat this process for all of the other tables. *Hint:* You can make a new table with the same schema as an existing table. For example:

```sql
CREATE TABLE 
      aus_men_2013 
(LIKE 
      aus_ladies_2013);
```

Extension: Can you make the tennis data tidy?

### Some practice SQL queries ###

The following SQL commands can be used to explore the data. To maximize understanding of the syntax, type these rather than copying and pasting.

In [1]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '54.149.231.104'
AWS_USERNAME = 'leo2506'
SSH_KEY_PATH = '/Users/liuriguang/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 56977


In [2]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'leo2506'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'leo2506' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'tennis'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [4]:
sql_query = '''SELECT 
      player1, player2, result 
      FROM 
          us_men_2013 
      LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Stephane Robert,Albano Olivetti,1
2,Jan-Lennard Struff,Guillaume Rufin,0
3,Aljaz Bedene,Dmitry Tursunov,0
4,Feliciano Lopez,Florent Serra,1


In [5]:
sql_query = '''
    SELECT
          player1, result 
    FROM 
          us_men_2013 
    WHERE 
          player1 = 'Richard Gasquet';
            '''

pd.read_sql_query(sql_query, cnx)

,player1,result
0,Richard Gasquet,1
1,Richard Gasquet,1
2,Richard Gasquet,1
3,Richard Gasquet,1
4,Richard Gasquet,1
5,Richard Gasquet,0


In [6]:
sql_query = '''
        SELECT 
              player1, player2, result
        FROM
              us_men_2013 
        WHERE 
              player1 = 'Richard Gasquet';
            '''

pd.read_sql_query(sql_query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Richard Gasquet,Stephane Robert,1
2,Richard Gasquet,Dmitry Tursunov,1
3,Richard Gasquet,Milos Raonic,1
4,Richard Gasquet,David Ferrer,1
5,Richard Gasquet,Rafael Nadal,0


In [7]:
sql_query = '''
SELECT
      player1, player2, result 
FROM 
      us_men_2013 
WHERE 
      player1 = 'Richard Gasquet' OR player2 = 'Richard Gasquet';
            '''

pd.read_sql_query(sql_query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Richard Gasquet,Stephane Robert,1
2,Richard Gasquet,Dmitry Tursunov,1
3,Richard Gasquet,Milos Raonic,1
4,Richard Gasquet,David Ferrer,1
5,Richard Gasquet,Rafael Nadal,0


In [8]:
sql_query = '''
SELECT 
      COUNT(*)
FROM
      us_men_2013;
            '''

pd.read_sql_query(sql_query, cnx)

,count
0,126


In [9]:
sql_query = '''
SELECT
      player1, COUNT(*) 
FROM 
      us_men_2013 
GROUP BY 
      player1;
            '''

pd.read_sql_query(sql_query, cnx).head()

,player1,count
0,Jiri Vesely,1
1,Tommy Haas,3
2,Sergiy Stakhovsky,1
3,Paolo Lorenzi,1
4,Jurgen Zopp,1


In [10]:
sql_query = '''
SELECT 
      player1, AVG(result) 
FROM 
      us_men_2013 
GROUP BY
      player1;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,avg
0,Jiri Vesely,0.000000
1,Tommy Haas,0.666667
2,Sergiy Stakhovsky,0.000000
3,Paolo Lorenzi,0.000000
4,Jurgen Zopp,0.000000
5,Ernests Gulbis,0.000000
6,Collin Altamirano,0.000000
7,Rajeev Ram,1.000000
8,Jarkko Nieminen,0.000000
9,John Isner,0.666667


In [11]:
sql_query = '''
SELECT 
      player1, player2, result 
FROM 
      us_men_2013 
WHERE 
      result = 1 
LIMIT 5;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,player2,result
0,Richard Gasquet,Michael Russell,1
1,Stephane Robert,Albano Olivetti,1
2,Feliciano Lopez,Florent Serra,1
3,Mikhail Kukushkin,Andrej Martin,1
4,Roberto Bautista Agut,Thomaz Bellucci,1


In [12]:
sql_query = '''
SELECT 
      COUNT(*) 
FROM 
      us_men_2013 
WHERE 
      result = 1;
            '''

pd.read_sql_query(sql_query, cnx)

,count
0,59


In [13]:
sql_query = '''
SELECT 
      player1, player2, result 
FROM 
      french_men_2013 
WHERE 
      result = 1 
LIMIT 5;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,player2,result
0,Somdev Devvarman,Daniel Munoz-De La Nava,1
1,Tobias Kamke,Paolo Lorenzi,1
2,Julien Benneteau,Ricardas Berankis,1
3,Jan Hajek,Denis Kudla,1
4,Gilles Simon,Lleyton Hewitt,1


In [14]:
sql_query = '''
SELECT 
      us_men_2013.player1, us_men_2013.tpw_1, french_men_2013.tpw_1 
FROM 
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,tpw_1,tpw_1
0,Tobias Kamke,62,140
1,Tobias Kamke,157,140
2,Julien Benneteau,63,163
3,Julien Benneteau,98,163
4,Julien Benneteau,125,163
5,Lukas Lacko,133,72
6,Adrian Mannarino,157,160
7,Benjamin Becker,125,85
8,Roberto Bautista Agut,104,100
9,Roberto Bautista Agut,87,100


In [15]:
sql_query = '''
SELECT 
      us_men_2013.player1, 
      SUM(us_men_2013.tpw_1) AS us_points, 
      SUM(french_men_2013.tpw_1) AS french_points
FROM 
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1 
GROUP BY 
      us_men_2013.player1;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,us_points,french_points
0,Jiri Vesely,112,120
1,Tommy Haas,1208,1569
2,Sergiy Stakhovsky,156,57
3,Jurgen Zopp,116,58
4,Ernests Gulbis,159,97
5,Jarkko Nieminen,138,172
6,Jan-Lennard Struff,149,132
7,Igor Sijsling,214,226
8,Lukas Lacko,133,72
9,Lukasz Kubot,86,90


In [16]:
sql_query = '''
SELECT
      us_men_2013.player1, 
      SUM(us_men_2013.tpw_1) AS us_points, 
      SUM(french_men_2013.tpw_1) AS french_points, 
      SUM(us_men_2013.tpw_1 + french_men_2013.tpw_1) AS total_points 
FROM
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1
GROUP BY
      us_men_2013.player1;
            '''

pd.read_sql_query(sql_query, cnx)

,player1,us_points,french_points,total_points
0,Jiri Vesely,112,120,232
1,Tommy Haas,1208,1569,2777
2,Sergiy Stakhovsky,156,57,213
3,Jurgen Zopp,116,58,174
4,Ernests Gulbis,159,97,256
5,Jarkko Nieminen,138,172,310
6,Jan-Lennard Struff,149,132,281
7,Igor Sijsling,214,226,440
8,Lukas Lacko,133,72,205
9,Lukasz Kubot,86,90,176


## The challenges!

This challenge uses only SQL queries. Please submit answers in a markdown file.

1. Using the same tennis data, find the number of matches played by
   each player in each tournament. (Remember that a player can be
   present as both player1 or player2).

In [ ]:
sql_command = '''
CREATE TABLE test AS

SELECT  player1 AS name,
        'M' AS gender,
        'US' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    us_men_2013

UNION ALL

SELECT  player2 AS name,
        'M' AS gender,
        'US' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    us_men_2013

UNION ALL

SELECT  player1 AS name,
        'M' AS gender,
        'AUS' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    aus_men_2013

UNION ALL

SELECT  player2 AS name,
        'M' AS gender,
        'AUS' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    aus_men_2013

UNION ALL

SELECT  player1 AS name,
        'M' AS gender,
        'French' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    french_men_2013

UNION ALL

SELECT  player2 AS name,
        'M' AS gender,
        'French' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    french_men_2013

UNION ALL

SELECT  player1 AS name,
        'M' AS gender,
        'wimbledon' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    wimbledon_men_2013

UNION ALL

SELECT  player2 AS name,
        'M' AS gender,
        'wimbledon' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    wimbledon_men_2013

UNION ALL

SELECT  player1 AS name,
        'F' AS gender,
        'wimbledon' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    wimbledon_ladies_2013

UNION ALL

SELECT  player2 AS name,
        'F' AS gender,
        'wimbledon' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    wimbledon_ladies_2013

UNION ALL

SELECT  player1 AS name,
        'F' AS gender,
        'French' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    french_ladies_2013

UNION ALL

SELECT  player2 AS name,
        'F' AS gender,
        'French' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    french_ladies_2013

UNION ALL

SELECT  player1 AS name,
        'F' AS gender,
        'AUS' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    aus_ladies_2013

UNION ALL

SELECT  player2 AS name,
        'F' AS gender,
        'AUS' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    aus_ladies_2013

UNION ALL

SELECT  player1 AS name,
        'F' AS gender,
        'US' AS tournament,
        result AS win,
        FSP_1 AS fsp,
        DBF_1 AS dbf,
        UFE_1 AS ufe
FROM    us_ladies_2013

UNION ALL

SELECT  player2 AS name,
        'F' AS gender,
        'US' AS tournament,
        1-result AS win,
        FSP_2 AS fsp,
        DBF_2 AS dbf,
        UFE_2 AS ufe
FROM    us_ladies_2013;
            '''
cnx.execute(sql_command)

In [19]:
sql_query = '''
SELECT * FROM test  
            '''

pd.read_sql_query(sql_query, cnx)

,name,gender,tournament,win,fsp,dbf,ufe
0,Richard Gasquet,M,US,1,63.0,7.0,NaN
1,Stephane Robert,M,US,1,61.0,2.0,NaN
2,Jan-Lennard Struff,M,US,0,55.0,13.0,NaN
3,Aljaz Bedene,M,US,0,52.0,8.0,NaN
4,Feliciano Lopez,M,US,1,58.0,3.0,NaN
5,Kenny De Schepper,M,US,0,59.0,11.0,NaN
6,Andrey Kuznetsov,M,US,0,53.0,8.0,NaN
7,Pablo Cuevas,M,US,0,51.0,6.0,NaN
8,Ernests Gulbis,M,US,0,58.0,8.0,NaN
9,Mikhail Kukushkin,M,US,1,51.0,5.0,NaN


In [20]:
sql_query = '''
    SELECT name, tournament, COUNT(*) AS matches
    FROM test
    GROUP BY name, tournament
    ORDER BY 1,2   
            '''

pd.read_sql_query(sql_query, cnx)

,name,tournament,matches
0,A Barty,US,1
1,A.Beck,wimbledon,2
2,A.Bedene,wimbledon,1
3,A.Bogomolov Jr.,wimbledon,1
4,A.Cadantu,wimbledon,2
5,A Cornet,US,3
6,A.Cornet,wimbledon,3
7,A.Dolgopolov,wimbledon,2
8,Adrian Mannarino,AUS,2
9,Adrian Mannarino,French,1


In [21]:
# pivot the answer
sql_query = '''
WITH count AS    
(SELECT name, tournament, COUNT(*) AS matches
    FROM test
    GROUP BY name, tournament
    ORDER BY 1,2)

SELECT name,
       SUM(CASE tournament WHEN 'US' THEN matches ELSE 0 END) AS US,
       SUM(CASE tournament WHEN 'AUS' THEN matches ELSE 0 END) AS AUS,
       SUM(CASE tournament WHEN 'French' THEN matches ELSE 0 END) AS French,
       SUM(CASE tournament WHEN 'wimbleton' THEN matches ELSE 0 END) AS wimbleton
FROM count
GROUP BY name
            '''

pd.read_sql_query(sql_query, cnx)

,name,us,aus,french,wimbleton
0,D.Kudla,0.0,0.0,0.0,0.0
1,J.Cepelova,0.0,0.0,0.0,0.0
2,F.Fognini,0.0,0.0,0.0,0.0
3,M.Russell,0.0,0.0,0.0,0.0
4,Julia Glushko,0.0,1.0,1.0,0.0
5,R.Dutra Silva,0.0,0.0,0.0,0.0
6,L Robson,3.0,0.0,0.0,0.0
7,Lesia Tsurenko,0.0,1.0,1.0,0.0
8,Michael Russell,1.0,1.0,1.0,0.0
9,Matthew Ebden,0.0,2.0,0.0,0.0


2. Who has played the most matches total in all of US Open, AUST Open, 
   French Open? Answer this both for men and women.

In [22]:
sql_query = '''
WITH most AS    
(SELECT name, gender, COUNT(*) AS matches 
FROM test
GROUP BY name, gender
ORDER BY 1,2)

SELECT * FROM
(SELECT *, RANK() OVER(PARTITION BY gender ORDER BY matches DESC) AS rank
FROM most) rank_model
WHERE rank = 1;
            '''

pd.read_sql_query(sql_query, cnx)

,name,gender,matches,rank
0,Agnieszka Radwanska,F,11,1
1,Maria Sharapova,F,11,1
2,Serena Williams,F,11,1
3,Victoria Azarenka,F,11,1
4,Rafael Nadal,M,21,1


3. Who has the highest first serve percentage? (Just the maximum value
   in a single match.)

In [23]:
# Identify no ties after observing the data
sql_query = '''
SELECT name, gender, tournament, fsp
FROM test
ORDER BY fsp DESC
LIMIT 1
            '''

pd.read_sql_query(sql_query, cnx)

,name,gender,tournament,fsp
0,S Errani,F,US,93.0


4. What are the unforced error percentages of the top three players
   with the most wins? (Unforced error percentage is % of points lost
   due to unforced errors. In a match, you have fields for number of
   points won by each player, and number of unforced errors for each
   field.)

In [24]:
sql_query = '''
WITH all_wins AS
(SELECT name, SUM(win) total_wins
FROM test
GROUP BY name
ORDER BY 2 DESC),
top3_wins AS
(SELECT * FROM 
    (SELECT *, RANK() OVER(ORDER BY total_wins DESC) rank 
    FROM all_wins) rank_model
WHERE rank <= 3)

SELECT top.name,rank, ROUND(SUM(ufe)*100.0/SUM(dbf+ufe),2) AS ufe_percentage 
FROM top3_wins top JOIN test
ON top.name = test.name
GROUP BY 1,2
ORDER BY 2
            '''

pd.read_sql_query(sql_query, cnx)

,name,rank,ufe_percentage
0,Rafael Nadal,1,93.60
1,Stanislas Wawrinka,2,91.98
2,Novak Djokovic,3,96.83
3,David Ferrer,3,90.89


*Hint:* `SUM(double_faults)` sums the contents of an entire column. For each row, to add the field values from two columns, the syntax `SELECT name, double_faults + unforced_errors` can be used.


*Special bonus hint:* To be careful about handling possible ties, consider using [rank functions](http://www.sql-tutorial.ru/en/book_rank_dense_rank_functions.html).

In [25]:
server.close()